In [1]:
%maven ai.djl:basicdataset:0.23.0
%maven ai.djl.mxnet:mxnet-engine:0.23.0
%maven org.slf4j:slf4j-simple:1.7.36

In [2]:
import ai.djl.*;
import ai.djl.engine.*;
import ai.djl.inference.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.Image;
import ai.djl.modality.cv.transform.*;
import ai.djl.modality.cv.translator.*;
import ai.djl.modality.cv.output.*;
import ai.djl.modality.cv.util.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.index.*;
import ai.djl.ndarray.index.NDIndex;
import ai.djl.ndarray.types.*;
import ai.djl.ndarray.types.Shape;
import ai.djl.nn.*;
import ai.djl.nn.convolutional.*;
import ai.djl.nn.core.*;
import ai.djl.nn.norm.*;
import ai.djl.nn.pooling.*;
import ai.djl.nn.recurrent.*;
import ai.djl.repository.zoo.*;
import ai.djl.training.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.initializer.*;
import ai.djl.training.listener.*;
import ai.djl.training.loss.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.tracker.*;
import ai.djl.training.util.*;
import ai.djl.training.dataset.*;
import ai.djl.translate.*;
import ai.djl.util.*;
import java.io.*;
import java.net.*;
import java.nio.*;
import java.nio.file.*;
import java.util.*;
import java.util.function.*;

In [3]:
String datasetLimit = System.getenv("DATASET_LIMIT");
if (datasetLimit != null) {
    System.setProperty("DATASET_LIMIT", datasetLimit);
}
String maxEpoch = System.getenv("MAX_EPOCH");
if (maxEpoch != null) {
    System.setProperty("MAX_EPOCH", maxEpoch);
}

System.setProperty("org.slf4j.simpleLogger.showThreadName", "false");
System.setProperty("org.slf4j.simpleLogger.showLogName", "false");
System.setProperty("org.slf4j.simpleLogger.log.ai.djl.pytorch", "WARN");
System.setProperty("org.slf4j.simpleLogger.log.ai.djl.mxnet", "WARN");
System.setProperty("org.slf4j.simpleLogger.log.ai.djl.tensorflow", "WARN");

In [4]:
%%loadFromPOM
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-jsplot</artifactId>
    <version>0.38.1</version>
</dependency>

In [6]:
import tech.tablesaw.api.*;
import tech.tablesaw.columns.*;
import tech.tablesaw.plotly.*;
import tech.tablesaw.plotly.api.*;
import tech.tablesaw.plotly.components.*;
import tech.tablesaw.plotly.traces.*;

In [7]:
%%loadFromPOM
<dependency>
    <groupId>com.opencsv</groupId>
    <artifactId>opencsv</artifactId>
    <version>5.9</version>
</dependency>


In [8]:
import com.opencsv.CSVReader;
import java.io.FileReader;
import java.io.IOException;

In [9]:
import java.util.ArrayList;
import java.util.List;

# Reading from a file

In [10]:
public static int[][] convertListToArray(List<int[]> dataList) {
        // function taking in List<int[]> and converting them into 2d int array (int[][])
        int rows = dataList.size();
        int cols = dataList.get(0).length; // Assuming all rows have the same length

        int[][] dataArray = new int[rows][cols];

        for (int i = 0; i < rows; i++) {
            dataArray[i] = dataList.get(i);
        }

        return dataArray;
    }

In [11]:
 public static int[] convertStringToIntArray(String inputString) {
        // function converting string of integers seperated by "," and turning it into integer array
        String[] stringArray = inputString.split(",");
        int[] intArray = new int[stringArray.length];

        for (int i = 0; i < stringArray.length; i++) {
            intArray[i] = Integer.parseInt(stringArray[i]);
        }

        return intArray;
    }

In [12]:
BufferedReader reader;
List<int[]> dataList = new ArrayList<>();
try {
	reader = new BufferedReader(new FileReader("C:\\Users\\Wiktor\\Desktop\\data\\train.txt"));
	String line = reader.readLine();
    int lineNumber = 0;
    int[] lineArray;
	while (line != null) {
        if(lineNumber>0)
        {
            lineArray = convertStringToIntArray(line);
            dataList.add(lineArray);
        }
        line = reader.readLine();
		lineNumber += 1;
	}

	reader.close();
} catch (IOException e) {
	e.printStackTrace();
}

In [13]:
dataList.size()

42000

In [14]:
int[][] data = convertListToArray(dataList);

In [15]:
int rows = data.length;
int cols = data[0].length;

System.out.println("Number of rows: " + rows);
System.out.println("Number of columns: " + cols);

Number of rows: 42000
Number of columns: 785


# Converting int[][] into NDArray

In [16]:
NDManager nd = NDManager.newBaseManager();

WARN No matching cuda flavor for win found: cu100mkl/sm_52.


In [17]:
NDArray dataND = nd.create(data);

In [18]:
dataND

ND: (42000, 785) cpu() int32
[ Exceed max print size ]

In [19]:
NDArray dataDev = dataND.get(new NDIndex("0:1000")).transpose();
dataDev

ND: (785, 1000) cpu() int32
[ Exceed max print size ]

# Spliting data into test and train samples

In [20]:
NDArray YDev = dataDev.get(0);
YDev

ND: (1000) cpu() int32
[ Exceed max print size ]

In [21]:
NDArray XDev = dataDev.get(new NDIndex("1:785"));
XDev

ND: (784, 1000) cpu() int32
[ Exceed max print size ]

In [22]:
XDev = XDev.div(255);
XDev

ND: (784, 1000) cpu() float32
[ Exceed max print size ]

In [23]:
NDArray dataTest = dataND.get(new NDIndex("1000:42000")).transpose();
dataTest

ND: (785, 41000) cpu() int32
[ Exceed max print size ]

In [24]:
NDArray YTest = dataTest.get(0);
YTest

ND: (41000) cpu() int32
[ Exceed max print size ]

In [25]:
NDArray XTest = dataTest.get(new NDIndex("1:785"));
XTest

ND: (784, 41000) cpu() int32
[ Exceed max print size ]

In [26]:
XTest = XTest.div(255);
XTest

ND: (784, 41000) cpu() float32
[ Exceed max print size ]

In [27]:
dataTest.get(0);

ND: (41000) cpu() int32
[ Exceed max print size ]

In [29]:
int rowTrain = (int) XTest.get(0).size();
rowTrain

41000

# Patameters Initialization

In [30]:
public static NDArray initializeW1()
{
    // we initialize W1 into a random integer from -0.5 to 0,5, its 10x784 because we have 10 digits and the digits are 28 x 28 pixels (28*28 = 784)
    NDArray W1 = nd.randomInteger(-500, 500, new Shape(10, 784), DataType.INT32);
    W1 = W1.div(1000);
    return W1;
}

In [31]:
public static NDArray initializeb1()
{
    // we initialize b1 into a random integer from -0.5 to 0,5,  itd 10x1 because the second layer has 10 nodes
    NDArray b1 = nd.randomInteger(-500, 500, new Shape(10, 1), DataType.INT32);
    b1 = b1.div(1000);
    return b1;
}

In [32]:
public static NDArray initializeW2()
{
    // we initialize W1 into a random integer from -0.5 to 0,5, its 10x10 because we have 10 digits and 10 nodes
    NDArray W2 = nd.randomInteger(-500, 500, new Shape(10, 10), DataType.INT32);
    W2 = W2.div(1000);
    return W2;
}

In [33]:
public static NDArray initializeb2()
{
    // we initialize b1 into a random integer from -0.5 to 0,5,  itd 10x1 because the second layer has 10 nodes
    NDArray b2 = nd.randomInteger(-500, 500, new Shape(10, 1), DataType.INT32);
    b2 = b2.div(1000);
    return b2;
}

# Functions declaration

In [34]:
public static NDArray ReLU(NDArray Z)
{
    // function returning ReLU activation function result (returns 0 if the data is smaller then 0, or the data if the data is >= 0)
    return Z.maximum(0);
}

In [35]:
public static NDArray softmax(NDArray Z)
{
    // function returning softmax activation function result (it calculates the probability of each function)
    return Z.exp().div(Z.exp().sum());
}

In [36]:
NDArray B = nd.arange(12);

B = B.div(100);

System.out.println(Arrays.toString(B.toFloatArray()));

B = B.softmax(0);

System.out.println(Arrays.toString(B.toFloatArray()));

B = B.cumSum();

System.out.println(Arrays.toString(B.toFloatArray()));

[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11]
[0.07882678, 0.079619, 0.08041918, 0.081227414, 0.08204376, 0.082868315, 0.083701156, 0.08454237, 0.085392036, 0.08625024, 0.08711707, 0.08799261]
[0.07882678, 0.15844578, 0.23886496, 0.32009238, 0.40213615, 0.48500445, 0.5687056, 0.653248, 0.73864007, 0.8248903, 0.9120074, 1.0]


In [37]:
public static NDArray forwardPropagateZ1(NDArray W1, NDArray X, NDArray b1)
{
    // Z = W * X + b, weight times value plus bias
    return W1.dot(X).add(b1);
}

In [38]:
public static NDArray forwardPropagateA1(NDArray Z1)
{
    // we use the ReLU activation function
    return ReLU(Z1);
}

In [39]:
public static NDArray forwardPropagateZ2(NDArray W2, NDArray A1, NDArray b2)
{
    // Z = W * X + b, weight times value plus bias
    return W2.dot(A1).add(b2);
}

In [40]:
public static NDArray forwardPropagateA2(NDArray Z2)
{
    // we use the softmax activation function
    return Z2.softmax(0);
}

In [41]:
public static NDArray ReLUDerivative(NDArray Z)
{
    return Z.gt(0);
}

# Backward Propagation 

In [42]:
public static NDArray backwardPropagateOneHotY(NDArray Y)
{
    // we generate one hot of Y to generate the target results
    return oneHot(Y);
}

In [43]:
public static NDArray backwardPropagateDZ2(NDArray A2, NDArray oneHotY)
{
    return A2.sub(oneHotY);
}

In [44]:
public static NDArray backwardPropagateDW2(double rows, NDArray DZ2, NDArray A1)
{
    double division = 1/rows;
    return DZ2.dot(A1.transpose()).mul(division);
}

In [45]:
public static NDArray backwardPropagateDb2(double rows, NDArray DZ2)
{
    return DZ2.sum().mul(1/rows);
}

In [46]:
public static NDArray backwardPropagateDZ1(NDArray W2, NDArray DZ2, NDArray Z1)
{
    return W2.transpose().dot(DZ2).mul(ReLUDerivative(Z1));
}

In [47]:
public static NDArray backwardPropagateDW1(double rows, NDArray DZ1, NDArray X)
{
    return DZ1.dot(X.transpose()).mul(1/rows);
}

In [48]:
public static NDArray backwardPropagateDb1(double rows, NDArray DZ1)
{
    return DZ1.sum().mul(1/rows);
}

In [49]:
public static NDArray updateParameter(NDArray param, double alpha, NDArray dParam, boolean isPrint)
{
    if(isPrint)
    {
            System.out.println(param.sub(dParam.mul(alpha)).get(new NDIndex("0:2, 0:2")));
    }
    return param.sub(dParam.mul(alpha));
}

In [50]:
public static NDArray getPredictions(NDArray A2)
{
    // returns the number with the biggest probabilities
    return A2.argMax(0).toType(DataType.INT32, false);
}

In [51]:
public static double getNumberOfCorrectPredictions(NDArray predictions, NDArray Y)
{
    // checks the predictions with the correct results and returns number of correct
    NDArray equalityMask = predictions.toType(DataType.INT32, false).eq(Y);
    return equalityMask.countNonzero().toType(DataType.FLOAT64, false).getDouble();
}

In [52]:
public static double getAccuracy(NDArray predictions, NDArray Y)
{
    // returns accuracy = number of correct predictions / all predictions
    System.out.println("Prediction: " + predictions.get(new NDIndex("0:10")) +", Desired output: " + Y.get(new NDIndex("0:10")));
    return getNumberOfCorrectPredictions(predictions, Y) / Y.size();
}

In [53]:
public static List<NDArray> mineGradientDescent(NDArray X, NDArray Y, double rows, double alpha, int iterations)
{
    double m = Y.size();
    NDArray Z1, A1, Z2, A2, dW1, dZ1, db1, dW2, dZ2, db2, oneHotY, predictions;
    double accuracy;
    NDArray W1 = initializeW1();
    NDArray b1 = initializeb1();
    NDArray W2 = initializeW2();
    NDArray b2 = initializeb2();
    for(int i = 0;i < iterations;i++)
    {
        Z1 = forwardPropagateZ1(W1, X, b1);
        A1 = forwardPropagateA1(Z1);
        Z2 = forwardPropagateZ2(W2, A1, b2);
        A2 = forwardPropagateA2(Z2);
        oneHotY = Y.oneHot(10).transpose();
        dZ2 = backwardPropagateDZ2(A2, oneHotY);
        dW2 = backwardPropagateDW2(m, dZ2, A1);
        db2 = backwardPropagateDb2(m, dZ2);
        dZ1 = backwardPropagateDZ1(W2, dZ2, Z1);
        dW1 = backwardPropagateDW1(m, dZ1, X);
        db1 = backwardPropagateDb1(m, dZ1);
        W1 = updateParameter(W1, alpha, dW1, false);
        b1 = updateParameter(b1, alpha, db1, false);
        W2 = updateParameter(W2, alpha, dW2, false);
        b2 = updateParameter(b2, alpha, db2, false);
        if((i+1)%10 == 0)
        {
            System.out.println("Iteration: " + (i+1));
            predictions = getPredictions(A2);
            accuracy = getAccuracy(predictions, Y);
            System.out.println("Accuracy: " + accuracy);
            System.out.println("-------------------");
        }
    }
    List<NDArray> result = new ArrayList<NDArray>();  
    result.add(W1);
    result.add(b1);
    result.add(W2);
    result.add(b2);
    return result;
}

In [54]:
List<NDArray> parameters = new ArrayList<NDArray>();
parameters = mineGradientDescent(XTest, YTest, rows, 0.1, 500);

Iteration: 10
Prediction: ND: (10) cpu() int32
[ 1,  0,  1,  1,  8,  6,  7,  2,  8,  2]
, Desired output: ND: (10) cpu() int32
[ 1,  5,  1,  7,  4,  8,  9,  5,  7,  4]

Accuracy: 0.17358536585365852
-------------------
Iteration: 20
Prediction: ND: (10) cpu() int32
[ 1,  0,  1,  1,  8,  8,  7,  2,  8,  2]
, Desired output: ND: (10) cpu() int32
[ 1,  5,  1,  7,  4,  8,  9,  5,  7,  4]

Accuracy: 0.2471219512195122
-------------------
Iteration: 30
Prediction: ND: (10) cpu() int32
[ 1,  0,  1,  1,  8,  8,  7,  2,  8,  2]
, Desired output: ND: (10) cpu() int32
[ 1,  5,  1,  7,  4,  8,  9,  5,  7,  4]

Accuracy: 0.29729268292682925
-------------------
Iteration: 40
Prediction: ND: (10) cpu() int32
[ 1,  0,  1,  1,  8,  8,  7,  2,  7,  2]
, Desired output: ND: (10) cpu() int32
[ 1,  5,  1,  7,  4,  8,  9,  5,  7,  4]

Accuracy: 0.3384390243902439
-------------------
Iteration: 50
Prediction: ND: (10) cpu() int32
[ 1,  6,  1,  1,  8,  8,  7,  2,  7,  2]
, Desired output: ND: (10) cpu() int32

In [55]:
public static NDArray makePredictions(NDArray X, NDArray W1, NDArray b1, NDArray W2, NDArray b2)
{
    NDArray Z1 = forwardPropagateZ1(W1, X, b1);
    NDArray A1 = forwardPropagateA1(Z1);
    NDArray Z2 = forwardPropagateZ2(W2, A1, b2);
    NDArray A2 = forwardPropagateA2(Z2);
    NDArray predictions = getPredictions(A2);
    return predictions;
}

In [59]:
NDArray predictions = makePredictions(XDev, parameters.get(0),  parameters.get(1), parameters.get(2), parameters.get(3));
double accuracy =  getAccuracy(predictions, YDev);
System.out.println("Accuracy:" + accuracy);

Prediction: ND: (10) cpu() int32
[ 1,  0,  1,  6,  0,  0,  7,  3,  5,  3]
, Desired output: ND: (10) cpu() int32
[ 1,  0,  1,  4,  0,  0,  7,  3,  5,  3]



Accuracy:0.828
